In [4]:
import sqlite3



In [40]:
db_name = "serenity_soother.db"

# get a connection to the database
conn = sqlite3.connect(db_name)

# get a cursor object
cursor = conn.cursor()

# retrieve all the tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# print the tables
print("Tables in the database:")
for table in tables:
    print(table[0])

# get the user table
cursor.execute("SELECT * FROM sections;")
users = cursor.fetchall()


Tables in the database:
users
sessions
hypnotherapy_scripts
emotions
goals
notifications
thoughts
memories
settings
trading_cards
achievements
rewards
scripts
hypnotherapy_sessions
progress
sections
scenes
elements
scene_element_association


In [41]:
len(users)  

16

In [49]:
users[8]

(9,
 1,
 '**Part2: Symbolism of Turquoise and Anxiety**',
 'Adorning this alcove is the striking hue of turquoise, representing a sanctuary of calm and communication. This color, with its restorative and serene essence, serves as a soothing counterbalance to the tumultuousness of anxiety, fostering an environment conducive to introspection.')